d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Upsert Into a Delta Table

**Objective:**  Repair records with an upsert

In the previous lesson, we identified two issues with the `health_tracker_processed` table:
- There were 72 missing records
- There were 60 records with broken readings

In this lesson, we will repair the table by modifying the `health_tracker_processed` table.

## Notebook Configuration

Before you run this cell, make sure to add a unique user name to the file
<a href="$./includes/configuration" target="_blank">
includes/configuration</a>, e.g.

```
username = "yourfirstname_yourlastname"
```

In [0]:
%run ./includes/configuration

Out[3]: DataFrame[]

In [0]:
%run ./includes/main/python/operations

## Prepare Updates DataFrame
To repair the broken sensor readings (less than zero), we'll interpolate using the value recorded before and after for each device. The Spark SQL functions LAG and LEAD will make this a trivial calculation.
We'll write these values to a temporary view called updates. This view will be used later to upsert values into our health_tracker_processed Delta table.

#### Step 1: Create a DataFrame Interpolating Broken Values
Recall that we want to partition on our Device ID column, which we named:
`"p_device_id"`

In [0]:

from pyspark.sql.window import Window
from pyspark.sql.functions import col, lag, lead

dteWindow = Window.partitionBy("p_device_id").orderBy("dte")

interpolatedDF = (
   spark.read
   .table("health_tracker_processed")
   .select(col("dte"),
           col("time"),
           col("heartrate"),
           lag(col("heartrate")).over(dteWindow).alias("prev_amt"),
           lead(col("heartrate")).over(dteWindow).alias("next_amt"),
           col("name"),
           col("p_device_id"))
 )

#### Step 2: Create a DataFrame of Updates

In [0]:
updatesDF = (
  interpolatedDF
  .where(col("heartrate") < 0)
  .select(col("dte"),
          col("time"),
          ((col("prev_amt") + col("next_amt"))/2).alias("heartrate"),
          col("name"),
          col("p_device_id"))
)


#### Step 3: View the schemas of the `updatesDF` and `health_tracker_processed` table
We use the `.printSchema()` function to view the schema of the `health_tracker_processed` table.

Fill in the format we should use and run the cell below.

In [0]:

(
  spark.read
  .format("delta")
  .load(health_tracker + "processed")
  .printSchema()
)
updatesDF.printSchema()

root
-- dte: date (nullable = true)
-- time: timestamp (nullable = true)
-- heartrate: double (nullable = true)
-- name: string (nullable = true)
-- p_device_id: integer (nullable = true)

root
-- dte: date (nullable = true)
-- time: timestamp (nullable = true)
-- heartrate: double (nullable = true)
-- name: string (nullable = true)
-- p_device_id: integer (nullable = true)

#### Step 4: Verify `updatesDF`
Perform a `.count()` on the `updatesDF` view. It should have the same number of records as the `SUM` performed on the broken_readings view.

In [0]:
updatesDF.count()

Out[10]: 60

## Prepare Inserts DataFrame
It turns out that our expectation of receiving the missing records late was correct. These records have subsequently been made available to us as the file `health_tracker_data_2020_02_01.json`.

#### Step 1: Load the Late-Arriving Data

In [0]:
file_path = health_tracker + "raw/late/health_tracker_data_2020_2_late.json"

health_tracker_data_2020_2_late_df = (
  spark.read
  .format("json")
  .load(file_path)
)


In [0]:

# Count the number of records
health_tracker_data_2020_2_late_df.count()

Out[14]: 72

#### Step 2: Transform the Data
In addition to updating the broken records, we wish to add this late-arriving data. We begin by preparing another temporary view with the appropriate transformations:
* Use the `from_unixtime` Spark SQL function to transform the unixtime into a time string
* Cast the `time` column to type `timestamp` to replace the column `time`
* Cast the `time` column to type `date` to create the column `dte`

In [0]:
insertsDF = process_health_tracker_data(spark, health_tracker_data_2020_2_late_df)

#### Step 3: View the Schema of the Inserts DataFrame

In [0]:
insertsDF.printSchema()


root
-- dte: date (nullable = true)
-- time: timestamp (nullable = true)
-- heartrate: double (nullable = true)
-- name: string (nullable = true)
-- p_device_id: integer (nullable = true)

#### Step 1: Create the Union DataFrame
Finally, we prepare the `upsertsDF` that consists of all the records in both the `updatesDF` and the `insertsDF`. We use the DataFrame `.union()` command to create the view.

In [0]:
upsertsDF = updatesDF.union(insertsDF)

#### Step 2: View the Schema

In [0]:
upsertsDF.printSchema()

root
-- dte: date (nullable = true)
-- time: timestamp (nullable = true)
-- heartrate: double (nullable = true)
-- name: string (nullable = true)
-- p_device_id: integer (nullable = true)

## Perform Upsert Into the `health_tracker_processed` Table
You can upsert data into a Delta table using the merge operation. This operation is similar to the SQL `MERGE` command but has added support for deletes and other conditions in updates, inserts, and deletes. In other words, using the DeltaTable command `.merge()` provides full support for an upsert operation.

#### Step 1: Perform the Upsert

In [0]:
from delta.tables import DeltaTable
processedDeltaTable = DeltaTable.forPath(spark, health_tracker + "processed")

update_match = """health_tracker.time = upserts.time
                  AND
                  health_tracker.p_device_id = upserts.p_device_id"""
update = {"heartrate" : "upserts.heartrate"}

insert = {
    "p_device_id" : "upserts.p_device_id",
    "heartrate" : "upserts.heartrate",
    "name" : "upserts.name",
    "time" : "upserts.time",
    "dte" : "upserts.dte"
}

(processedDeltaTable.alias("health_tracker")
 .merge(upsertsDF.alias("upserts"), update_match)
 .whenMatchedUpdate(set=update)
 .whenNotMatchedInsert(values=insert)
 .execute())

## View the Commit Using Time Travel

#### Step 1: View the table as of Version 2
This is done by specifying the option `"versionAsOf"` as 2. When we time travel to Version 0, we see only the first month of data. In version 1, we see the table after we added comments. 
When we time travel to Version 2, we see the first two months of data, minus the 72 missing records.

In [0]:
(spark.read
 .option("versionAsOf", 2)
 .format("delta")
 .load(health_tracker + "processed")
 .count())

Out[20]: 7128

#### Step 2: Count the Most Recent Version
When we query the table without specifying a version, it shows the latest version of the table and includes the full two months of data.
Note that the range of data includes the month of February during a leap year. That is why there are 29 days in the month.

In [0]:
spark.read.table("health_tracker_processed").count()

Out[21]: 7200

#### Step 3: Describe the History of the `health_tracker_processed` Table
The `.history()` Delta Table command provides provenance information, including the operation, user, and so on, for each write to a table.
Note that each operation performed on the table is given a version number. These are the numbers we have been using when performing a time travel query on the table, e.g., `SELECT COUNT(*) FROM health_tracker_processed VERSION AS OF 1`.

In [0]:
display(processedDeltaTable.history())

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
3,2021-07-06T18:35:38.000+0000,6324118605225795,mscianika@gmail.com,MERGE,"Map(predicate -> ((health_tracker.`time` = upserts.`time`) AND (health_tracker.`p_device_id` = upserts.`p_device_id`)), matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(2261997875604044),0706-163832-mutes431,2,WriteSerializable,false,"Map(numTargetRowsCopied -> 7068, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 5, executionTimeMs -> 16819, numTargetRowsInserted -> 72, scanTimeMs -> 4142, numTargetRowsUpdated -> 60, numOutputRows -> 7200, numTargetChangeFilesAdded -> 0, numSourceRows -> 132, numTargetFilesRemoved -> 10, rewriteTimeMs -> 12609)",null
2,2021-07-06T17:52:18.000+0000,6324118605225795,mscianika@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(2261997875603935),0706-163832-mutes431,1,WriteSerializable,true,"Map(numFiles -> 5, numOutputBytes -> 53042, numOutputRows -> 3408)",null
1,2021-07-06T17:48:59.000+0000,6324118605225795,mscianika@gmail.com,REPLACE COLUMNS,"Map(columns -> [{""name"":""dte"",""type"":""date"",""nullable"":true,""metadata"":{""comment"":""Format: YYYY/mm/dd""}},{""name"":""time"",""type"":""timestamp"",""nullable"":true,""metadata"":{}},{""name"":""heartrate"",""type"":""double"",""nullable"":true,""metadata"":{}},{""name"":""name"",""type"":""string"",""nullable"":true,""metadata"":{""comment"":""Format: First Last""}},{""name"":""p_device_id"",""type"":""integer"",""nullable"":true,""metadata"":{""comment"":""range 0 - 4""}}])",null,List(2261997875603972),0706-163832-mutes431,0,SnapshotIsolation,true,Map(),null
0,2021-07-06T17:47:29.000+0000,6324118605225795,mscianika@gmail.com,CONVERT,"Map(numFiles -> 5, partitionedBy -> [""p_device_id""], collectStats -> true)",null,List(2261997875603972),0706-163832-mutes431,-1,Serializable,false,Map(numConvertedFiles -> 5),null


## Perform a Second Upsert
In the previous lesson, we performed an upsert to the `health_tracker_processed` table, which updated records containing broken readings. When we inserted the late arriving data, we inadvertently added more broken readings!

#### Step 1: Sum the Broken Readings

In [0]:

from pyspark.sql.functions import col, count

broken_readings = (
  spark.read
  .format("delta")
  .load(health_tracker + "processed")
  .select(col("heartrate"), col("dte"))
  .where(col("heartrate") < 0)
  .groupby("dte")
  .agg(count("heartrate"))
  .orderBy("dte")
)
broken_readings.createOrReplaceTempView("broken_readings")


#### Step 2: Verify That These are New Broken Readings
Let’s query the broken_readings with a `WHERE` clause to verify that these are indeed new broken readings introduced by inserting the late-arriving data.
Note that there are no broken readings before ‘2020-02-25’.

In [0]:
%sql
SELECT SUM(`count(heartrate)`) FROM broken_readings WHERE dte < '2020-02-25'

sum(count(heartrate))
null


#### Step 3: Verify Updates
Perform a `.count()` on the `updatesDF` view.

**Note:** It is not necessary to redefine the DataFrame. Recall that a Spark DataFrame is lazily defined, pulling the correct number of updates when an action is triggered.
It should have the same number of records as the SUM performed on the `broken_readings` view.

In [0]:

updatesDF.count()

Out[26]: 1

#### Step 4: Perform Upsert Into the `health_tracker_processed` Table
Once more, we upsert into the `health_tracker_processed` table using the DeltaTable command `.merge()`.

In [0]:
upsertsDF = updatesDF

(processedDeltaTable.alias("health_tracker")
 .merge(upsertsDF.alias("upserts"), update_match)
 .whenMatchedUpdate(set=update)
 .whenNotMatchedInsert(values=insert)
 .execute())

#### Step 5: Sum the Broken Readings
Let’s sum the records in the `broken_readings` view one last time.

In [0]:
%sql
SELECT SUM(`count(heartrate)`) FROM broken_readings


sum(count(heartrate))
null


-sandbox
&copy; 2021 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>